In [ ]:
!pip install yfinance ipywidgets matplotlib pandas

import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display
from datetime import datetime

def download_stock_data(ticker, start_date, end_date):
    try:
        data = yf.download(ticker, start=start_date, end=end_date)
        if data.empty:
            print(f"No data found for {ticker}")
            return None
        return data['Close']
    except Exception as e:
        print(f"Error downloading data for {ticker}: {e}")
        return None

def calculate_daily_returns(prices):
    return prices.pct_change() * 100

def plot_stock_comparison(stock1, stock2, start_date, end_date):
    prices1 = download_stock_data(stock1, start_date, end_date)
    prices2 = download_stock_data(stock2, start_date, end_date)

    if prices1 is None or prices2 is None:
        return

    returns1 = calculate_daily_returns(prices1)
    returns2 = calculate_daily_returns(prices2)

    plt.figure(figsize=(14, 7))

    plt.plot(returns1.index, returns1, label=f'{stock1} Daily Return', alpha=0.7)
    plt.plot(returns2.index, returns2, label=f'{stock2} Daily Return', alpha=0.7)

    plt.axhline(0, color='gray', linestyle='--', linewidth=0.5)

    plt.title(f'Daily Returns Comparison: {stock1} vs {stock2}\n{start_date} to {end_date}')
    plt.xlabel('Date')
    plt.ylabel('Daily Return (%)')
    plt.legend()
    plt.grid(True, alpha=0.3)

    plt.xticks(rotation=45)

    plt.tight_layout()
    plt.show()

    print("\nStatistics:")
    print(f"{stock1} - Average daily return: {returns1.mean():.2f}%")
    print(f"{stock2} - Average daily return: {returns2.mean():.2f}%")
    print(f"{stock1} - Volatility (std dev): {returns1.std():.2f}%")
    print(f"{stock2} - Volatility (std dev): {returns2.std():.2f}%")

stock1_input = widgets.Text(
    value='AAPL',
    placeholder='Enter stock ticker (e.g., AAPL)',
    description='Stock 1:',
    disabled=False
)

stock2_input = widgets.Text(
    value='MSFT',
    placeholder='Enter stock ticker (e.g., MSFT)',
    description='Stock 2:',
    disabled=False
)

start_date_picker = widgets.DatePicker(
    description='Start Date:',
    value=datetime(2020, 1, 1),
    disabled=False
)

end_date_picker = widgets.DatePicker(
    description='End Date:',
    value=datetime(2023, 12, 31),
    disabled=False
)

compare_button = widgets.Button(
    description='Compare Stocks',
    button_style='success',
    tooltip='Click to compare'
)

output = widgets.Output()

def on_compare_button_clicked(b):
    with output:
        output.clear_output()
        start_date = start_date_picker.value.strftime('%Y-%m-%d')
        end_date = end_date_picker.value.strftime('%Y-%m-%d')
        plot_stock_comparison(stock1_input.value.upper(),
                             stock2_input.value.upper(),
                             start_date,
                             end_date)

compare_button.on_click(on_compare_button_clicked)

input_box = widgets.HBox([stock1_input, stock2_input])
date_box = widgets.HBox([start_date_picker, end_date_picker])
button_box = widgets.HBox([compare_button])
main_box = widgets.VBox([input_box, date_box, button_box, output])

display(main_box)